In [1]:
%pip install -r "../requirements2.txt"

Note: you may need to restart the kernel to use updated packages.


### Celda 1: Definir y Limpiar los Documentos

Este código descarga un archivo desde GitHub, lo guarda temporalmente, lo carga y limpia su contenido eliminando caracteres especiales y etiquetas HTML innecesarias.

In [2]:
import requests
from langchain_community.document_loaders import TextLoader
from dotenv import load_dotenv
import os

# Cargar las variables de entorno desde el archivo .env
load_dotenv()

# Usar root como base del GitHub
root = os.getenv("ROOT")
if not root:
    raise ValueError("ROOT no encontrado en el archivo .env")

print("Root de GitHub:", root)

# URL del archivo en GitHub
file_path = f"{root}/documentos_y_matcomplement/docuentreno/md/Cholesterol-Myths-vs-Facts-Spanish.md"
print("URL del archivo:", file_path)

# Descargar el contenido del archivo
response = requests.get(file_path)
if response.status_code != 200:
    raise RuntimeError(f"Error al descargar el archivo desde {file_path}")

# Guardar el contenido en un archivo temporal
with open("temp.md", "w", encoding="utf-8") as temp_file:
    temp_file.write(response.text)

# Cargar el documento .md desde el archivo temporal
text_loader = TextLoader("temp.md")
documents = text_loader.load()

# Limpiar el contenido del documento
def clean_text(text):
    # Eliminar caracteres especiales no deseados
    text = text.replace('\ufeff', '')
    text = text.replace('<a name="br1"></a>', '')
    return text.strip()

# Procesar y limpiar el contenido del documento
cleaned_documents = []
for doc in documents:
    cleaned_text = clean_text(doc.page_content)
    cleaned_documents.append(cleaned_text)

# Mostrar el contenido limpio del documento
for doc in cleaned_documents:
    print("Contenido limpio del documento:", doc)

Root de GitHub: https://raw.githubusercontent.com/davidfdezmartin/Chatbot-con-FastAPI-Streamlit-y-LangChain/main
URL del archivo: https://raw.githubusercontent.com/davidfdezmartin/Chatbot-con-FastAPI-Streamlit-y-LangChain/main/documentos_y_matcomplement/docuentreno/md/Cholesterol-Myths-vs-Facts-Spanish.md
Contenido limpio del documento: COLESTEROL: MITOS Y VERDADES

Mito: Sólo necesitará analizar sus niveles de colesterol cuando tenga más de 30 años.

Verdad: La Asociación Americana del Corazón (AHA) recomienda analizar los niveles de colesterol

una vez entre los 9 y 11 años, y de nuevo entre los 17 y 21 años para niños y adultos jóvenes sin otros

factores de riesgos o un historial familiar de enfermedad cardíaca temprana. Después de los 20

años, su doctor volverá a analizar su colesterol y otros factores de riesgo cada cuatro a seis años

siempre y cuando su riesgo se mantenga bajo.

Mito: Sólo la gente con sobrepeso u obesidad tiene colesterol alto.

Verdad: Las personas con cualq

### Celda 2: Crear los Embeddings

Este código inicializa un modelo de embeddings utilizando `HuggingFaceEmbeddings`, genera embeddings para cada documento limpio y los imprime para su análisis.

In [3]:
from langchain.embeddings import HuggingFaceEmbeddings

# Embedding Using Huggingface
huggingface_embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",  # Puedes cambiar esto a "sentence-transformers/all-MiniLM-l6-v2" si lo prefieres
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

# Crear embeddings para el contenido limpio del documento
embeddings = []
for doc in cleaned_documents:
    embedding = huggingface_embeddings.embed_documents([doc])
    embeddings.append(embedding)

# Imprimir los embeddings para análisis
for idx, embedding in enumerate(embeddings):
    print(f"Embeddings del documento {idx + 1}: {embedding}")

d:\Users\casti\pinokio\bin\miniconda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\Users\casti\pinokio\bin\miniconda\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Embeddings del documento 1: [[0.006454624235630035, -0.024622324854135513, 0.02791258506476879, -0.003843842539936304, 0.017961403355002403, -0.02698516845703125, 0.03887294605374336, 0.015038227662444115, -0.05206865817308426, -0.10043297708034515, -0.019520960748195648, -0.05342983454465866, -0.033275384455919266, -0.020517688244581223, 0.03118455782532692, -0.054445914924144745, 0.015190445818006992, 0.017293337732553482, -0.07992972433567047, 0.01815580390393734, 0.05100220814347267, 0.0016113464953377843, -0.0034621753729879856, -0.03114328533411026, 0.007098376285284758, -0.0158835481852293, 0.007994985207915306, -0.004058383405208588, -0.031081916764378548, -0.11827611178159714, -0.02185015007853508, -0.03272420912981033, 0.04569700360298157, -0.06815893948078156, -0.026118846610188484, -0.008991883136332035, 0.002970912028104067, 0.023290090262889862, 0.0283779576420784, 0.018596313893795013, 0.05692153796553612, 0.05050521343946457, 0.003344488563016057, 0.004964482970535755, 

### Celda 3: Crear Embeddings de una Consulta

Utiliza el método `embed_query` para crear embeddings a partir del contenido del primer documento limpio.
Convierte los embeddings a un array de NumPy y muestra el array y su forma.

In [4]:
import numpy as np

# Crear embeddings para una consulta específica del primer documento limpio
query_embedding = huggingface_embeddings.embed_query(cleaned_documents[0])

# Mostrar los embeddings como un array de NumPy y su forma
print(np.array(query_embedding))
print(np.array(query_embedding).shape)

[ 6.45462424e-03 -2.46223249e-02  2.79125851e-02 -3.84384254e-03
  1.79614034e-02 -2.69851685e-02  3.88729461e-02  1.50382277e-02
 -5.20686582e-02 -1.00432977e-01 -1.95209607e-02 -5.34298345e-02
 -3.32753845e-02 -2.05176882e-02  3.11845578e-02 -5.44459149e-02
  1.51904458e-02  1.72933377e-02 -7.99297243e-02  1.81558039e-02
  5.10022081e-02  1.61134650e-03 -3.46217537e-03 -3.11432853e-02
  7.09837629e-03 -1.58835482e-02  7.99498521e-03 -4.05838341e-03
 -3.10819168e-02 -1.18276112e-01 -2.18501501e-02 -3.27242091e-02
  4.56970036e-02 -6.81589395e-02 -2.61188466e-02 -8.99188314e-03
  2.97091203e-03  2.32900903e-02  2.83779576e-02  1.85963139e-02
  5.69215380e-02  5.05052134e-02  3.34448856e-03  4.96448297e-03
  5.46228960e-02 -6.91622868e-02 -8.16256776e-02 -5.15112691e-02
 -3.48759088e-04  2.43555997e-02 -1.76036786e-02  2.53083147e-02
 -3.69075947e-02  4.04026210e-02  2.99918223e-02 -2.76719895e-03
  1.13067608e-02  4.28193249e-03  2.28792932e-02  4.69351001e-02
  4.61886302e-02  5.41891

### Celda 4: Creación del VectorStore

Utiliza `FAISS` para crear un `VectorStore` a partir de los documentos finales y los embeddings generados, incluyendo metadatos.

In [5]:
from langchain.vectorstores import FAISS
from langchain.schema import Document  # Importar la clase Document desde langchain.schema

# Crear objetos de documento adecuados a partir de las cadenas de texto limpias con metadatos
final_documents = [Document(page_content=content, metadata={"source": f"Documento {i+1}"}) for i, content in enumerate(cleaned_documents[:120])]

# Crear el VectorStore utilizando FAISS y los embeddings generados
vectorstore = FAISS.from_documents(final_documents, huggingface_embeddings)

# Verificación del VectorStore creado
print("VectorStore creado con éxito:", vectorstore)

VectorStore creado con éxito: <langchain_community.vectorstores.faiss.FAISS object at 0x000001EC1D9DA5C0>


### Celda 5: Realizar una Consulta Usando Búsqueda por Similaridad

Esta celda utiliza el `VectorStore` para realizar una búsqueda por similaridad basándose en una consulta específica. Luego, imprime el contenido del documento más relevante encontrado.

In [6]:
## Query using Similarity Search
query = "TENGO ALTO EL COLESTEROL QUE DEBO HACER, TENGO 35 AÑOS"
relevant_documents = vectorstore.similarity_search(query)

print(relevant_documents[0].page_content)

COLESTEROL: MITOS Y VERDADES

Mito: Sólo necesitará analizar sus niveles de colesterol cuando tenga más de 30 años.

Verdad: La Asociación Americana del Corazón (AHA) recomienda analizar los niveles de colesterol

una vez entre los 9 y 11 años, y de nuevo entre los 17 y 21 años para niños y adultos jóvenes sin otros

factores de riesgos o un historial familiar de enfermedad cardíaca temprana. Después de los 20

años, su doctor volverá a analizar su colesterol y otros factores de riesgo cada cuatro a seis años

siempre y cuando su riesgo se mantenga bajo.

Mito: Sólo la gente con sobrepeso u obesidad tiene colesterol alto.

Verdad: Las personas con cualquier tipo de cuerpo pueden tener colesterol alto. Tener sobrepeso

u obesidad aumenta sus probabilidades de tener colesterol alto, pero ser delgado no lo protege.

Independientemente de su peso, dieta y nivel de actividad física, debería medir su nivel de

colesterol con regularidad.

Mito: Tener colesterol alto es un problema sólo de lo

### Celda 6: Configurar el Retriever

Configura un `Retriever` a partir del `VectorStore` utilizando búsqueda por similaridad y muestra su configuración.

In [7]:
## Configurar el Retriever
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# Realizar una consulta usando el Retriever
query = "TENGO ALTO EL COLESTEROL QUE DEBO HACER, TENGO 35 AÑOS"
relevant_documents = retriever.get_relevant_documents(query)

# Imprimir el contenido relevante y sus metadatos
for idx, doc in enumerate(relevant_documents):
    print(f"Documento relevante {idx + 1}:")
    print(f"Fuente: {doc.metadata['source']}")
    print(f"Contenido: {doc.page_content}\n")

Documento relevante 1:
Fuente: Documento 1
Contenido: COLESTEROL: MITOS Y VERDADES

Mito: Sólo necesitará analizar sus niveles de colesterol cuando tenga más de 30 años.

Verdad: La Asociación Americana del Corazón (AHA) recomienda analizar los niveles de colesterol

una vez entre los 9 y 11 años, y de nuevo entre los 17 y 21 años para niños y adultos jóvenes sin otros

factores de riesgos o un historial familiar de enfermedad cardíaca temprana. Después de los 20

años, su doctor volverá a analizar su colesterol y otros factores de riesgo cada cuatro a seis años

siempre y cuando su riesgo se mantenga bajo.

Mito: Sólo la gente con sobrepeso u obesidad tiene colesterol alto.

Verdad: Las personas con cualquier tipo de cuerpo pueden tener colesterol alto. Tener sobrepeso

u obesidad aumenta sus probabilidades de tener colesterol alto, pero ser delgado no lo protege.

Independientemente de su peso, dieta y nivel de actividad física, debería medir su nivel de

colesterol con regularidad.


d:\Users\casti\pinokio\bin\miniconda\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
